# Destination polling

In this notebook we show how to deploy an interactive jupyter notebook (like this one) using Voila and Heroku. We were inspired by [Martin Renou's repository](https://github.com/martinRenou/voila_heroku) so we follow closely his steps for deployment, while adding interactivity to the notebook with the use of ipywidgets and ipyleaflet libraries.

## App objective

The purpose of this app is to poll attendees to an event, e.g. a meeting, and display the results of the polling. Also, we would like to showcase some of the possibilities of using notebooks and voila for creating useful simple apps. 

In this example the attendee is presented with the question: Where would you like to go for vacation?
After typing the desired destination, we send a geocoding request to LocationIQ using the geocoder library to get a list of potential locations.

If we receive a valid response from the request we present the user with the returned list (up to four) of places that match the desired destination on a Select widget. By default, we show on a ipyleaflet Map a Marker located on the coordinates of the first returned location. This Marker position will change based on the user selection from the Selection widget.

Once the user has selected the desired location we store it and then we present the user with the current results of the polling on a map, with one circle (CircleMarker) for each different destination entered, varying the circle radius base on the total number of votes (popularity) for that particular destination. 

## LocationIQ setup

The are a lot of [geocoding providers](https://geocoder.readthedocs.io/index.html#providers) supported by the geocoder library. Our LocationIQ choice was made based on [minimal restrictions and setup requirements](https://locationiq.com/pricing) for a free account.

### Restrictions

As our geocoding requests will be submitted by people and not machines, we can comply with the first 3 restrictions without too much trouble. For the last three restrictions we annotate the reason it doesn't apply to us:  

1. 10,000 requests /day 
2. 60 requests /minute 
3. 2 requests /second 
4. Street maps only: We won't use this.
5. Limited commercial use: Our tool is not commercial.
6. Cache results for upto 48 hours: Free Heroku app resets after 30 min. of idleness. 


### Setup requirements

First setup your [LocationIQ free account](https://locationiq.com/register). You will only need a valid email address. Just follow the steps on the website.

After successful registration you will be provided with a key. This key must be stored in the text file (key.txt) next to this notebook's location. 


## Heroku setup

Get a free Heroku account and follow this instructions to [install Heroku CLI](https://devcenter.heroku.com/articles/getting-started-with-python#set-up) on your machine.


## Dependencies

We need to install these libraries the current environment before executing the next cell:

1. [ipywidget](https://ipywidgets.readthedocs.io/en/latest/user_install.html):
    `conda install -c conda-forge ipywidgets`

2. [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/installation.html):
`conda install -c conda-forge ipyleaflet`

3. [geocoder](https://geocoder.readthedocs.io/api.html#installation):
`pip install geocoder`

4. [voila](https://github.com/QuantStack/voila):
`conda install voila -c conda-forge`

In [1]:
import time
import json
from ipyleaflet import Map, CircleMarker, Marker, FullScreenControl, WidgetControl, basemaps, basemap_to_tiles
from ipywidgets import HTML, Button, Text, Select

import geocoder

import warnings

warnings.filterwarnings("ignore")

# conda install -c conda-forge ipywidgets
# https://ipywidgets.readthedocs.io/en/latest/user_install.html

# conda install -c conda-forge ipyleaflet
# https://ipyleaflet.readthedocs.io/en/latest/installation.html

# pip install geocoder
# https://geocoder.readthedocs.io/api.html#installation

# conda install voila -c conda-forge
# https://github.com/QuantStack/voila

# Run once: Save initial JSON

## TODO: Remove in final version

In [2]:
if False:
    # Save this
    cities = {'214334866': {'lat': 40.7127281,
      'lon': -74.0060152,
      'address': 'New York City, New York, USA',
      'count': 1}}

    with open("cities.json", "w") as fp:
        json.dump(cities, fp)

# User interface

## Widgets

First we define the widgets to be used in the map (ipyleaflet):

- input_city: A Text field for the user to input the name of a place.

- geocode_input_city: A button to process the place typed in the input_city text field. This will trigger a geocoding query. If successful present geocoding results in the select widget, set the select option index to the first element, and then place a marker on the map tied to this option's coordinates; else invite user to input a valid place.

- select: A single selection from options widget. Selection options are based on the geocoding results. The index will default to the first place returned by the geocoding. If the option is changed, the current marker will be remove, and a new marker will be added to the map.

- vote: Add the current selected option to the database (JSON file). Then display circle markers to show all entries to date, with the circle size proportional to the place's popularity (total number of entries or count). Then, deactivate all widgets to prevent the user from submitting multiple votes, and activate the refresh button, so the user can get and updated view after casting his/her vote.

In [3]:
input_city = Text(
                value="Tahiti",
                placeholder="Type the name of a place",
                description="Where to?",
                disabled=False
            )

geocode_input_city = Button(
                description="Find places",
                disabled=False,
                button_style="success", # 'success', 'info', 'warning', 'danger' or ''
                tooltip="Click me",
                icon='check'
            )

#style = {"description_width": "initial"}
select = Select(
            options=[""],
            value= "",
            # rows=10,
            description="Select a city:",
            disabled=False,
           # style=style
            )


vote = Button(
              description="Vote!",
              disabled=True,
              button_style="success", # 'success', 'info', 'warning', 'danger' or ''
              tooltip="Click me",
              icon='check'
            )

refresh = Button(
              description="Refresh map",
              disabled=False,
              button_style="info", # 'success', 'info', 'warning', 'danger' or ''
              tooltip="Click me",
              icon='check'
            )

## Map

For geocoding we will use [LocationIQ](https://locationiq.com/). A key is required to use their API, to monitor fair use of their service. To get a free key you will only need a valid email address. Free account use restrictions are:

- 10,000 requests /day 
- 60 requests /minute 
- 2 requests /second 
- Street maps only
- Limited commercial use
- Cache results for upto 48 hours

As our use case is well within these restrictions we considered this a reasonable compromise.

In [4]:
with open("key.txt", "r") as fhandle:
    key = fhandle.readline().split()[0]

In [5]:
def on_geocode_input_city_cliked(button):
    
    # TODO: How to pass around the geocoded object? using global variable for now
    global geocoded_cities
    geocoded_cities = geocoder.locationiq(input_city.value, key=key, maxRows=4)
    
    if geocoded_cities.ok:
        
        select.options = [city.address for city in geocoded_cities]
        
        if geocode_input_city.button_style != "success":
            geocode_input_city.button_style = "success"
        
        select.index = 0
        
        vote.disabled=False
        
    else:
        select.options = ["Please try again"]
        geocode_input_city.button_style = "danger"
        geocode_input_city.description = "Invalid city name"
        
        vote.disabled=True

In [6]:
# revert button style to success, after a bad city input
def on_input_text_selection(change):
    geocode_input_city.button_style = "success"
    geocode_input_city.description = "Find places"
    
    select.options = [""]
    
input_city.observe(on_input_text_selection, names="value")

In [7]:
def remove_map_layer_type(m, layer_type):
    for layer in m.layers:
        if layer.__class__.__name__ == layer_type:
            m.remove_layer(layer)

In [8]:
def on_selection_change(change):
    
    if geocoded_cities.ok:
        remove_map_layer_type(m, "Marker")

        city_idx = select.index
        selected_city = geocoded_cities[city_idx]

        name = selected_city.address
        lat = selected_city.lat
        long = selected_city.lng

        marker = Marker(location=[lat, long])
        m.add_layer(marker)

        message = HTML()
        message.value = name

        marker.popup = message
    
select.observe(on_selection_change, names="index")    

In [9]:
def refresh_plot(m, cities=None):
    
    # read cities stored in db (past votes)
    if cities == None:
        with open("cities.json", "r") as fp:
            cities = json.load(fp)
    
    # Remove old circles if present
    remove_map_layer_type(m, "CircleMarker")
    
    # Get max_count to normalize the circle sizes
    max_count = 0
    for city in cities:
        count = cities[city]["count"]
        if max_count < count:
            max_count = count
    
    # Plot circles for each city with radius proportinal to count (number of votes)
    max_radius = 20
    for city in cities:
        name = cities[city]["address"].split(",")[0]
        count = cities[city]["count"]
        lat = cities[city]["lat"]
        long = cities[city]["lon"]

        radius = round((count * max_radius)/ max_count)
        marker = CircleMarker(location=[lat, long], fill_color='red', color='red', radius=radius, weight=2)
        m.add_layer(marker)
        marker.popup = HTML(f"{name}, Popularity: {count}")

In [10]:
def on_vote_clicked(button):
    
    # remove selection markers
    remove_map_layer_type(m, "Marker")
    
    # read cities stored in db (past votes)
    with open("cities.json", "r") as fp:
        cities = json.load(fp)
    
    city_idx = select.index
    selected_city = geocoded_cities[city_idx]
    
    # add to count if city in cities, else add city to dict
    if geocoded_cities[city_idx].place_id in cities:
        cities[selected_city.place_id]["count"] += 1
    else:
        cities[selected_city.place_id] = {"lat": selected_city.lat,
                                          "lon": selected_city.lng,
                                          "address": selected_city.address,
                                          "count": 1}
    # save db
    with open("cities.json", "w") as fp:
        json.dump(cities, fp)
        
    select.disabled = True
    vote.disabled = True
    geocode_input_city.disabled = True
    input_city.disabled = True
    
    refresh_plot(m, cities=cities)
    
    # Remove Widget controls after succesful vote
    for control in m.controls:
        if control.__class__.__name__ == "WidgetControl":
            m.remove_control(control)
    
    # Add refresh button
    widget_control_refresh = WidgetControl(widget=refresh, position="bottomleft")
    m.add_control(widget_control_refresh)

In [11]:
def on_refresh_clicked(button):
    refresh_plot(m)

In [12]:
m = Map(zoom=2, world_copy_jump=True)

# TODO: Constrain to only 1 or 1.5 background map

m.add_control(FullScreenControl())

widget_control_inputText = WidgetControl(widget=input_city, position="bottomleft")
widget_control_geocode_input_city = WidgetControl(widget=geocode_input_city, position="bottomleft")
widget_control_select = WidgetControl(widget=select, position="bottomleft")
widget_control_vote = WidgetControl(widget=vote, position="bottomleft")

m.add_control(widget_control_vote)
m.add_control(widget_control_select)
m.add_control(widget_control_geocode_input_city)
m.add_control(widget_control_inputText)

geocode_input_city.on_click(on_geocode_input_city_cliked)

vote.on_click(on_vote_clicked)

refresh.on_click(on_refresh_clicked)

m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

Status code 404 from https://locationiq.org/v1/search.php: ERROR - 404 Client Error: Not Found for url: https://locationiq.org/v1/search.php?key=3ba299e90244b4&q=xxyy&format=json&addressdetails=1&limit=4
